<a href="https://colab.research.google.com/github/NicoEssi/Data_Science_Portfolio/blob/master/Stackoverflow_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Setup

In [0]:
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# Downloading the Stack Overflow Survey Results for 2019
!wget --no-check-certificate \
    "https://drive.google.com/uc?authuser=0&id=1QOmVDpd8hcVYqqUXDXf68UMDWQZP0wQV&export=download" \
    -O "/tmp/soi_2019.zip"
zip_ref = zipfile.ZipFile("/tmp/soi_2019.zip", 'r')
zip_ref.extractall("/tmp/soi_2019")
zip_ref.close()

--2019-09-11 00:58:51--  https://drive.google.com/uc?authuser=0&id=1QOmVDpd8hcVYqqUXDXf68UMDWQZP0wQV&export=download
Resolving drive.google.com (drive.google.com)... 173.194.69.101, 173.194.69.100, 173.194.69.138, ...
Connecting to drive.google.com (drive.google.com)|173.194.69.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://drive.google.com/uc?id=1QOmVDpd8hcVYqqUXDXf68UMDWQZP0wQV&export=download [following]
--2019-09-11 00:58:56--  https://drive.google.com/uc?id=1QOmVDpd8hcVYqqUXDXf68UMDWQZP0wQV&export=download
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-14-4c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qj4kgl4em0jopo83p40ghujs4mu282f5/1568160000000/06716978924947585995/*/1QOmVDpd8hcVYqqUXDXf68UMDWQZP0wQV?e=download [following]
--2019-09-11 00:58:59--  https://doc-14-4c-docs.googleusercontent.com/docs/securesc

# 1. Business- & Data Understanding

In [0]:
data = pd.read_csv("/tmp/soi_2019/survey_results_public.csv")
schema = pd.read_csv("/tmp/soi_2019/survey_results_schema.csv")

In [0]:
#for i in range(len(schema)):
#    print(schema.iloc[i].Column + " : " + schema.iloc[i].QuestionText)

In [5]:
print("US Count: " + str(np.sum(data.Country == "United States")))
print("UK Count: " + str(np.sum(data.Country == "United Kingdom")))
print("EU Count: " + str(np.sum(data.Country == "France")
                        + np.sum(data.Country == "Germany")
                        + np.sum(data.Country == "Sweden")
                        + np.sum(data.Country == "Denmark")
                        + np.sum(data.Country == "Finland")
                        + np.sum(data.Country == "Ireland")
                        + np.sum(data.Country == "Netherlands")
                        + np.sum(data.Country == "Austria")
                        + np.sum(data.Country == "Belgium")
                        + np.sum(data.Country == "Switzerland")))

US Count: 20949
UK Count: 5737
EU Count: 15591


In [4]:
print("Before: " + str(len(data.CompTotal)))

data = data[pd.notnull(data['CompFreq'])]
data = data[pd.notnull(data['CompTotal'])].reset_index(drop = True)

print("After: " + str(len(data.CompTotal)))

Before: 88883
After: 55827


In [7]:
print("US Count: " + str(np.sum(data.Country == "United States")))
print("UK Count: " + str(np.sum(data.Country == "United Kingdom")))
print("EU Count: " + str(np.sum(data.Country == "France")
                        + np.sum(data.Country == "Germany")
                        + np.sum(data.Country == "Sweden")
                        + np.sum(data.Country == "Denmark")
                        + np.sum(data.Country == "Finland")
                        + np.sum(data.Country == "Ireland")
                        + np.sum(data.Country == "Netherlands")
                        + np.sum(data.Country == "Austria")
                        + np.sum(data.Country == "Belgium")
                        + np.sum(data.Country == "Switzerland")
                        + np.sum(data.Country == "Norway")))

US Count: 14981
UK Count: 4036
EU Count: 10744


## We are interested in identifying the profiles of the top earners in each region; United States, United Kingdom, and Europe. Thereafter, we'll build a model to predict salary.

### Why do we conduct our inquiry on two countries, yet one is of a larger region consisting of numerous countries (Europe)?
Professionals residing within the European Union have liberties granted to them by the Schengen agreement, which enables free movement of labor force and thus professionals can freely work wherever they wish.

### Why are Norway and Switzerland included in the European dataset?
While they are not part of the European Union, they have signed agreements in association with the Schengen agreement.

### Why is the United Kingdom not included in the European dataset despite having signed an agreement in association with the Schengen agreement?
Brexit. Despite the fact that the data has been gathered prior to the secession deadline, it would still be interesting to inquire on the UK data separately in case it would be of interest to compare changes in 2020 and onwards. And as of writing this, nothing has been officially said regarding future signed agreement in association with the Schengen agreement for UK.

### Why are the countries in APT / RCEP not included?
Unfortunately the data gathered for China, South Korea, and Japan are too small to make reliable inferences - and there are currently significant restrictions in freedom of movement for professionals between these countries, making it unreasonable to conduct an inquiry on these countries as a collective.

# 2. Data Preparation

In [5]:
#del data['Respondent']

data = data[data["MainBranch"] == 'I am a developer by profession']
data = data[data["LastHireDate"] != "I've never had a job"]

len(data)

50397

In [55]:
data.EdLevel.unique()

array(['Bachelor’s degree (BA, BS, B.Eng., etc.)',
       'Some college/university study without earning a degree',
       'Master’s degree (MA, MS, M.Eng., MBA, etc.)',
       'Other doctoral degree (Ph.D, Ed.D., etc.)',
       'Primary/elementary school', 'Associate degree',
       'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)',
       nan, 'Professional degree (JD, MD, etc.)',
       'I never completed any formal education'], dtype=object)

In [0]:
relevant = ["Hobbyist", "OpenSourcer", "Employment", "Country",
"Student", "EdLevel", "UndergradMajor", "EduOther", "OrgSize",
"DevType", "YearsCode", "YearsCodePro", "JobSeek", "LastHireDate",
"FizzBuzz", "CompTotal", "CompFreq", "CurrencySymbol",
"LanguageWorkedWith", "DatabaseWorkedWith", "PlatformWorkedWith", "WebFrameWorkedWith", "MiscTechWorkedWith"]

data_us = data[data["Country"] == "United States"]
data_us = data_us[relevant]

data_uk = data[data["Country"] == "United Kingdom"]
data_uk = data_uk[relevant]

data_eu = data[data["Country"].isin(["France", "Germany", "Sweden",
                                     "Denmark", "Finland", "Norway",
                                     "Ireland", "Netherlands", "Austria",
                                     "Belgium", "Switzerland"])]
data_eu = data_eu[relevant]

data_test = data[data["Country"] == "Canada"]
data_test = data_test[relevant]

In [0]:
# currency rates as of 09/09/2019
"""
EUR = 1.10
CHF = 1.00
SEK = 0.10
NOK = 0.11
DKK = 0.15
USD = 1.00
GBP = 1.23
INR = 0.014
NZD = 0.64
AUD = 0.69
CAD = 0.76
"""

currencies = {"EUR" : 1.10
             ,"CHF" : 1.00
             ,"SEK" : 0.10
             ,"NOK" : 0.11
             ,"DKK" : 0.15
             ,"USD" : 1.00
             ,"GBP" : 1.23
             ,"INR" : 0.014
             ,"NZD" : 0.64
             ,"AUD" : 0.69
             ,"CAD" : 0.76}

# Convert currency into USD
def currency_usd(data):
    if data in (currencies):
        return currencies[data]
    else:
        return 0

# Annualize total compensation
def currency_annualize(data):
    if data == 'Weekly':
        return 52
    elif data == 'Monthly':
        return 12
    elif data == 'Yearly':
        return 1

In [0]:
# Parse all types within a data into a list
def parse_types(data):
    types_temp = []
    for i in range( len(data)):
        types_temp.append([parsed.strip() for parsed in str(data.iloc[i]).split(';')])
    
    types = []
    for i in types_temp:
        for j in i:
            if j not in types:
                types.append(j)
                
    return types

In [0]:
# Parse datarow of all text into a list
def parse(data):
    return [parsed.strip() for parsed in str(data).split(';')]

# Check whether text exists in data
def check(text, data):
    if text in parse(data):
        return 1
    else:
        return 0

# Count occurences of text in data
def countd(text, data):
    counted = []
    
    for i in data:
        counted.append(check(text, i))
    
    return sum(counted)

In [0]:
devtypes = parse_types(data.DevType)
eduother = parse_types(data.EduOther)
languages = parse_types(data.LanguageWorkedWith)
databases = parse_types(data.DatabaseWorkedWith)
platforms = parse_types(data.PlatformWorkedWith)
webframes = parse_types(data.WebFrameWorkedWith)
misctechs = parse_types(data.MiscTechWorkedWith)

In [0]:
data_us["CompTotal"] *= data_us["CompFreq"].apply(currency_annualize)
data_us["CompTotal"] *= data_us["CurrencySymbol"].apply(currency_usd)
data_us = data_us[data_us.CompTotal > 100]

data_uk["CompTotal"] *= data_uk["CompFreq"].apply(currency_annualize)
data_uk["CompTotal"] *= data_uk["CurrencySymbol"].apply(currency_usd)
data_uk = data_uk[data_uk.CompTotal > 100]

data_eu["CompTotal"] *= data_eu["CompFreq"].apply(currency_annualize)
data_eu["CompTotal"] *= data_eu["CurrencySymbol"].apply(currency_usd)
data_eu = data_eu[data_eu.CompTotal > 100]

## To do list:


*   One hot encode categorical variables
*   Validate data
*   Model the data
*   Results
*   Deploy



In [23]:
# This may take a while. Will refactor to speed up process.
for i in range( len(data_test.DevType)):
    data_test.DevType.iloc[i] = parse(data_test.DevType.iloc[i])

# Update: No need to parse. Use data_test["DevType"].str.get_dummies(sep=";")

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [37]:
data_test["DevType"].str.get_dummies(sep=";")

,Academic researcher,Data or business analyst,Data scientist or machine learning specialist,Database administrator,Designer,DevOps specialist,"Developer, QA or test","Developer, back-end","Developer, desktop or enterprise applications","Developer, embedded applications or devices","Developer, front-end","Developer, full-stack","Developer, game or graphics","Developer, mobile",Educator,"Engineer, data","Engineer, site reliability",Engineering manager,Marketing or sales professional,Product manager,Scientist,Senior executive/VP,Student,System administrator
66,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0
75,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
140,0,0,1,1,1,0,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1
153,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
164,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
184,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0
185,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
198,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
302,0,0,0,0,0,1,0,1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0
346,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
